In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np
from scipy.spatial.distance import euclidean, pdist, squareform
from os.path import join as os_join
import seaborn as sns
import matplotlib.pylab as plt
from pyvis import network as net


pd.set_option('display.max_colwidth', None)

In [2]:
similarity_path = "../data/processed/vis_dataset/similarity_doc2vec.csv"
dataset_path = "../data/processed/vis_dataset/vis_data.csv"

output_path = "../reports/figures/doc2vec_similarity/similarity_Doc2Vec.html"

In [3]:
dataset = pd.read_csv(dataset_path)

dataset["Title"] = dataset["Title"].astype(str)
dataset["AuthorNames"] = dataset["AuthorNames"].astype(str)


dataset.sample(2)

,Conference,Year,Title,DOI,id,cite_to_list,cited_by_list,Link,FirstPage,LastPage,...,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_02-2020,XploreCitationCount - 2020-01,PubsCited,Award
1441,VAST,2017,Graphiti: Interactive Specification of Attribute-Based Edges for Network Modeling and Visualization,10.1109/TVCG.2017.2744843,1441,"[641, 290, 257, 293, 678, 453, 267, 1329, 471, 604]",[1586],http://dx.doi.org/10.1109/TVCG.2017.2744843,226,235,...,"Network visualizations, often in the form of node-link diagrams, are an effective means to understand relationships between entities, discover entities with interesting characteristics, and to identify clusters. While several existing tools allow users to visualize pre-defined networks, creating these networks from raw data remains a challenging task, often requiring users to program custom scripts or write complex SQL commands. Some existing tools also allow users to both visualize and model networks. Interaction techniques adopted by these tools often assume users know the exact conditions for defining edges in the resulting networks. This assumption may not always hold true, however. In cases where users do not know much about attributes in the dataset or when there are several attributes to choose from, users may not know which attributes they could use to formulate linking conditions. We propose an alternate interaction technique to model networks that allows users to demonstrate to the system a subset of nodes and links they wish to see in the resulting network. The system, in response, recommends conditions that can be used to model networks based on the specified nodes and links. In this paper, we show how such a demonstration-based interaction technique can be used to model networks by employing it in a prototype tool, Graphiti. Through multiple usage scenarios, we show how Graphiti not only allows users to model networks from a tabular dataset but also facilitates updating a pre-defined network with additional edge types.",Arjun Srinivasan;Hyunwoo Park;Alex Endert;Rahul C. Basole,Arjun Srinivasan;Hyunwoo Park;Alex Endert;Rahul C. Basole,Georgia Institute of Technology;The Ohio State University;Georgia Institute of Technology;Georgia Institute of Technology,10.1109/VAST.2006.261429;10.1109/TVCG.2011.185;10.1109/VAST.2011.6102441;10.1109/VAST.2006.261426;10.1109/TVCG.2009.151;10.1109/TVCG.2006.122;10.1109/TVCG.2016.2598839;10.1109/TVCG.2006.166;10.1109/TVCG.2009.108;10.1109/VAST.2010.5652520,"Network modeling,visual analytics,user interaction",1.0,1.0,49.0,NaN
472,InfoVis,2009,Interactive Dimensionality Reduction Through User-defined Combinations of Quality Metrics,10.1109/TVCG.2009.153,472,"[390, 265, 171, 206, 148, 244, 216, 185, 59]","[595, 598, 644, 662, 674, 684, 794, 828, 829, 859, 897, 904, 1041, 1051, 1126, 1153, 1190, 1205, 1281, 1335, 1369, 1555]",http://dx.doi.org/10.1109/TVCG.2009.153,993,1000,...,"Multivariate data sets including hundreds of variables are increasingly common in many application areas. Most multivariate visualization techniques are unable to display such data effectively, and a common approach is to employ dimensionality reduction prior to visualization. Most existing dimensionality reduction systems focus on preserving one or a few significant structures in data. For many analysis tasks, however, several types of structures can be of high significance and the importance of a certain structure compared to the importance of another is often task-dependent. This paper introduces a system for dimensionality reduction by combining user-defined quality metrics using weight functions to preserve as many important structures as possible. The system aims at effective visualization and exploration of structures within large multivariate data sets and provides enhancement of diverse structures by supplying a range of automatic variable orderings. Furthermore it enables a quality-guided reductio

In [4]:
id_title_abstract = "<b>Paper id</b>: " + dataset["id"].astype(str) + " ("  + dataset["Conference"] + ") " " - "\
"<a href=\""  + dataset.Link + "\" target=\"_blank\">Link</a>" + \
"; <br><b>Authors: </b>: " + dataset["AuthorNames"] + \
"; <br><b>Title</b>: " + dataset["Title"] + \
"; <br><b>Abstract</b>: " + dataset["Abstract"]

map_id_to_label={k: v for  k,v in enumerate(id_title_abstract)}
map_id_to_label[0]

'<b>Paper id</b>: 0 (InfoVis)  - <a href="http://dx.doi.org/10.1109/INFVIS.1995.528680" target="_blank">Link</a>; <br><b>Authors: </b>: B. Spence;L. Tweedie;H. Dawkes;Hua Su; <br><b>Title</b>: Visualisation for functional design; <br><b>Abstract</b>: We present two novel visualisation tools: the Influence Explorer and the Prosection Matrix. These were specifically created to support engineering artifact design and similar tasks in which a set of parameter values must be chosen to lead to acceptable artifact performance. These tools combine two concepts. One is the interactive and virtually immediate responsive display of data in a manner conducive to the acquisition of insight. The other, involving the precalculation of samples of artifact performance, facilitates smooth exploration and optimisation leading to a design decision. The anticipated benefits of these visualisation tools are illustrated by an example taken from electronic circuit design, in which full account must be taken o

In [5]:
df = pd.read_csv(similarity_path)
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745
0,0,-1.000000,0.568328,0.462271,0.454161,0.642155,0.540096,0.551160,0.468066,0.390690,...,0.355781,0.456573,0.568745,0.630027,0.402972,0.444379,0.558462,0.557408,0.560330,0.496985
1,1,0.568328,-1.000000,0.327876,0.514606,0.296113,0.381139,0.490727,0.506431,0.262349,...,0.541579,0.507317,0.338627,0.570750,0.223580,0.119695,0.601685,0.377666,0.311815,0.488700
2,2,0.462271,0.327876,-1.000000,0.528790,0.258828,0.613659,0.638728,0.544126,0.383845,...,0.288476,0.524038,0.333307,0.262148,0.579781,0.486939,0.251883,0.420083,0.374725,0.461788
3,3,0.454161,0.514606,0.528790,-1.000000,0.500015,0.569363,0.540131,0.480091,0.479014,...,0.515064,0.421636,0.511353,0.379185,0.534273,0.527216,0.442845,0.702163,0.657056,0.439307
4,4,0.642155,0.296113,0.258828,0.500015,-1.000000,0.553881,0.645605,0.158116,0.642725,...,0.341039,0.323165,0.467028,0.343945,0.418254,0.549289,0.367514,0.668831,0.550037,0.438186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,1741,0.444379,0.119695,0.486939,0.527216,0.549289,0.515795,0.546503,0.379432,0.466030,...,0.352275,0.191376,0.514101,0.318249,0.593109,-1.000000,0.452709,0.589822,0.691349,0.522740
1742,1742,0.558462,0.601685,0.251883,0.442845,0.367514,0.264349,0.447632,0.403387,0.271778,...,0.614482,0.455766,0.656377,0.409905,0.269491,0.452709,-1.000000,0.496920,0.534461,0.421971
1743,1743,0.557408,0.377666,0.420083,0.702163,0.668831,0.549612,0.687164,0.272261,0.595197,...,0.403439,0.452550,0.710840,0.381544,0.669028,0.589822,0.496920,-1.000000,0.797513,0.552322
1744,1744,0.560330,0.311815,0.374725,0.657056,0.550037,0.490000,0.615911,0.308767,0.421654,...,0.576391,0.356563,0.710266,0.339426,0.571327,0.691349,0.534461,0.797513,-1.000000,0.475081


In [6]:
fill_diagonal = -1
thres_to_plot = 0.855

    
arr = df.values
index_names = df.index
col_names = df.columns

#  Get indices where such threshold is crossed; avoid diagonal elems
R,C = np.where(np.triu(arr,1)>=thres_to_plot)

# Arrange those in columns and put out as a dataframe
out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
df_out = pd.DataFrame(out_arr,columns=['from','to','value'])
df_out["from"] = df_out["from"].astype(np.int16)
df_out["to"] = df_out["to"].astype(np.int16)
df_out

,from,to,value
0,15,658,0.863793
1,15,951,0.861554
2,18,434,0.864562
3,21,754,0.878708
4,21,1453,0.891444
...,...,...,...
949,1632,1699,0.855051
950,1666,1667,0.89219
951,1678,1713,0.879727
952,1682,1725,0.856969


## Plot using Pyvis

In [7]:
color_dict = {
    "InfoVis": "blue",
    "VAST": "orange",
    "SciVis": "red"
}

def get_node_color(node, df, color_dict):
    conference = df[df.id == node].Conference.values[0]
    return color_dict[conference]

In [8]:
thres = thres_to_plot
df_out_thres = df_out[df_out["value"] >= thres]
print("df_out_thres.shape", df_out_thres.shape)
plotting_all_nodes = False

file_name= output_path
# g = net.Network(height='95%', width='99%',heading='Paper Similarity Using #BibliographicCouplings and #Co-citations')

g = net.Network(height="700px", width="100%",heading='Paper Similarity Using Doc2Vec')
g.set_template("html_files/template_2.html")

g.set_edge_smooth('discrete')
# g.set_options('{"nodes": {"borderWidthSelected": "10" } }')

if plotting_all_nodes:
    node_list = list(df.index.astype(np.int16))
else:
    node_list = list(set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) )

node_labels = [ map_id_to_label.get(node) for node in node_list]
node_colors = [ get_node_color(node, dataset, color_dict) for node in node_list]
    
# g.add_nodes(node_list, title=node_labels, label=node_list, color=node_colors)

g.add_nodes(node_list, title=node_labels, color=node_colors)


edge_list = zip(df_out_thres["from"], df_out_thres["to"], df_out_thres['value'])
g.add_edges(edge_list)    
g.show_buttons(filter_=["physics", "edges"])
# g.show_buttons()

g.set_options("""
var options =   {
        "physics": { "stabilization": false ,
            "barnesHut" : {"centralGravity":0.1,     "gravitationalConstant":-5000}
                    },
 "edges": { "smooth": { "type": "continuous" } } 
 
}
 
  """)

g.show(file_name)

#  source code: https://github.com/WestHealth/pyvis/blob/523d541d04df6779dd21e5942af60b79369e8f13/pyvis/network.py#L218

df_out_thres.shape (954, 3)


In [21]:

node = g.nodes[0]
looking_node = node

neighbors = g.neighbors(node["id"])
subnetwork_ids = list(neighbors) + [looking_node["id"]]
subnetwork_edges = list()

if neighbors:
    for node in g.nodes:
        if node["id"] == looking_node["id"]:
            node.update({"size": "60"})
        elif node["id"] in neighbors:
            pass
        else:
            node.update({"color": "gray"})


    for edge in g.edges:
        if edge['from'] in subnetwork_ids and edge["to"] in subnetwork_ids:
            subnetwork_edges.append(edge)
    g_sub = net.Network(height="700px", width="100%",heading='Sub Network')
    g_sub.set_template("html_files/template_2.html")
    g_sub.add_nodes(subnetwork_ids)
    g_sub.add_edges([ [e['from'], e["to"], e["width"]] for e in subnetwork_edges]) 

    g_sub.show("html_files/small/sub_network.html")




FileNotFoundError: [Errno 2] No such file or directory: 'html_files/small/sub_network.html'

In [33]:
import dill
    
with open('/Users/chaupham/minhchau/MY_WORK/C/citation_networks/src/streamlit_network/models/doc2vec_similarity.pkl', 'wb') as f:     
    dill.dump(g, f)
    
with open('/Users/chaupham/minhchau/MY_WORK/C/citation_networks/src/streamlit_network/dict/node_colors.pkl', 'wb') as f:     
    dill.dump(g, f)

In [ ]:
# !open "/Users/chaupham/minhchau/MY_WORK/C/citation_networks/reports/figures/doc2vec_similarity"

In [32]:
!open $file_name

In [ ]:
!open .

## Plot using Jaal

More elegant and faster, but still in its infancy though. There's a lack of some crucial features such as "edge thickness"

In [ ]:
dataset

In [ ]:
import pandas as pd
from jaal import Jaal
from jaal.datasets import load_got


thres = 6
plotting_all_nodes = False


    
    

df_out_thres = df_out[df_out["value"] >= thres]
edges = df_out_thres

(dataset)
# edges.loc[:, 'label'] = edges.loc[:, 'value'].astype(int).astype(str)

edges["from"] = edges["from"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0]))
edges["to"] = edges["to"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0])) 


set_thres_nodes = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) )
nodes = pd.DataFrame({"id": df.index})
nodes["having_edge"] = nodes["id"].map(lambda x: x in set_thres_nodes).astype(str)
nodes["conference"] = nodes["id"].map(lambda node_id: dataset[dataset["id"] == node_id ]["Conference"].values[0])
nodes["Title"] = nodes["id"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0])) 
nodes["id"] = nodes["Title"]
if plotting_all_nodes:
    pass
else:
    print(nodes.shape)
    node_list = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) 
    nodes = nodes[nodes["id"].isin(node_list)]
    print(nodes.shape)

nodes.to_csv("../data/interim/jaal_data/nodes_2.csv", index=False)
edges.to_csv("../data/interim/jaal_data/edges_2.csv", index=False)

port=8050

while True:
    try:
        Jaal(edges, nodes).plot(port=port)
    except:
        port+=1


In [ ]:
edges["value"] = edges["value"].astype(int)

In [ ]:
edges.nlargest(10,"value")

In [ ]:
node_list

In [ ]:
nodes[nodes.id==1606].Title.values[0]

In [ ]:
edges[(edges["to"] == 755) & (edges["from"]==456)]